# Gradient Descent Optimizers
### Applying various algorithms that attempt to improve gradient descent
Based on this overview: https://ruder.io/optimizing-gradient-descent/

In [1]:
import numpy as np
import pandas as pd

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [2]:
ddf = pd.read_csv('Data/digits_train.csv')

In [3]:
X = ddf.drop(['label'], axis=1).to_numpy()
digit_y = ddf[['label']].to_numpy()

In [4]:
X = X/255

In [5]:
vector_y = [[0]*10 for i in range(digit_y.shape[0])]

for i in range(digit_y.shape[0]):
    vector_y[i][digit_y[i][0]] = 1.0
    
y = np.array(vector_y)

In [6]:
all_data = np.concatenate((X,y), axis=1)

In [7]:
np.random.shuffle(all_data)
train = all_data[:10000]
test = all_data[41000:]

In [8]:
class NeuralNet:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
        
    def train(self, training_data, time, learning_rate, batch_size, test_data=None):   
        training_size = len(training_data)
        self.rate = learning_rate
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):
        rolling_delta = y - self.activations[-1]        
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            delta_b = rolling_delta * derivative
            delta_w = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.biases[i] += self.rate*np.sum(delta_b, axis=0)
            self.weights[i] += self.rate*delta_w            
            
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)
        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [50]:
no_opt = NeuralNet([784, 50, 30, 10])
no_opt.train(train, 30, .001, 50, test)

epoch 1: 11.1% accurate
epoch 2: 10.1% accurate
epoch 3: 13.4% accurate
epoch 4: 13.0% accurate
epoch 5: 13.1% accurate
epoch 6: 13.8% accurate
epoch 7: 14.8% accurate
epoch 8: 16.0% accurate
epoch 9: 17.5% accurate
epoch 10: 18.5% accurate
epoch 11: 20.9% accurate
epoch 12: 22.0% accurate
epoch 13: 23.4% accurate
epoch 14: 25.4% accurate
epoch 15: 26.9% accurate
epoch 16: 28.3% accurate
epoch 17: 30.5% accurate
epoch 18: 32.7% accurate
epoch 19: 34.8% accurate
epoch 20: 37.5% accurate
epoch 21: 39.0% accurate
epoch 22: 41.5% accurate
epoch 23: 42.6% accurate
epoch 24: 44.7% accurate
epoch 25: 46.3% accurate
epoch 26: 48.4% accurate
epoch 27: 49.6% accurate
epoch 28: 51.4% accurate
epoch 29: 52.7% accurate
epoch 30: 53.8% accurate


## Momentum
* Uses previous delta along with current delta
* If one gradient keeps moving in the same direction that will change more than a gradient that is changing directions

In [10]:
class Momentum:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        # adding self variables for the weights/biases deltas
        # since we are tracking them over time now
        self.weights_delta = []
        self.biases_delta = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.weights_delta.append(np.zeros((layers[i-1], layers[i])))          
            self.biases_delta.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, learning_rate, batch_size, moment, test_data=None):   
        training_size = len(training_data)
        self.rate = learning_rate
        
        # add a momentum term 
        self.momentum = moment
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]        
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            bias_grad = rolling_delta * derivative
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.biases_delta[i] = self.momentum*self.biases_delta[i] + self.rate*np.sum(bias_grad, axis=0)
            self.weights_delta[i] = self.momentum*self.weights_delta[i] + self.rate*weight_grad  
            
            self.biases[i] += self.biases_delta[i]
            self.weights[i] += self.weights_delta[i]     
            
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [47]:
moment = Momentum([784, 50, 30, 10])
moment.train(train, 30, .001, 50, .9, test)

epoch 1: 27.9% accurate
epoch 2: 36.4% accurate
epoch 3: 42.9% accurate
epoch 4: 45.7% accurate
epoch 5: 47.3% accurate
epoch 6: 49.1% accurate
epoch 7: 50.1% accurate
epoch 8: 51.6% accurate
epoch 9: 53.5% accurate
epoch 10: 55.8% accurate
epoch 11: 57.7% accurate
epoch 12: 60.1% accurate
epoch 13: 63.1% accurate
epoch 14: 65.2% accurate
epoch 15: 65.7% accurate
epoch 16: 67.7% accurate
epoch 17: 69.1% accurate
epoch 18: 70.4% accurate
epoch 19: 75.8% accurate
epoch 20: 78.4% accurate
epoch 21: 78.8% accurate
epoch 22: 79.5% accurate
epoch 23: 80.5% accurate
epoch 24: 80.7% accurate
epoch 25: 81.3% accurate
epoch 26: 82.2% accurate
epoch 27: 82.8% accurate
epoch 28: 83.0% accurate
epoch 29: 84.3% accurate
epoch 30: 84.7% accurate


## Nesterov Accelerated Gradient (NAG)
* builds on the momentum integration
* we know our parameters are going to move by the momentum/previous delta term
* so we apply the previous delta to the parameters before calculating the gradient
* that way if the momentum would overshoot us we can correct for it

In [12]:
class NAG:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        # adding self variables for the weights/biases deltas
        # since we are tracking them over time now
        self.weights_delta = []
        self.biases_delta = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.weights_delta.append(np.zeros((layers[i-1], layers[i])))          
            self.biases_delta.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, learning_rate, batch_size, moment, test_data=None):   
        training_size = len(training_data)
        self.rate = learning_rate
        
        # add a momentum term 
        self.momentum = moment
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x, training=True)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x, training):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):
            # use the previous delta to calculate the activations (unless using test data)
            nag_weights = self.weights[i] + self.weights_delta[i] if training else self.weights[i]
            nag_biases = self.biases[i] + self.biases_delta[i] if training else self.biases[i]
            
            z = np.dot(self.activations[-1], nag_weights) + nag_biases
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            bias_grad = rolling_delta * derivative
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.biases_delta[i] = self.momentum*self.biases_delta[i] + self.rate*np.sum(bias_grad, axis=0)
            self.weights_delta[i] = self.momentum*self.weights_delta[i] + self.rate*weight_grad  
            
            self.biases[i] += self.biases_delta[i]
            self.weights[i] += self.weights_delta[i]
            
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]], training=False)
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [46]:
nag = NAG([784, 50, 30, 10])
nag.train(train, 30, .001, 50, .9, test)

epoch 1: 20.4% accurate
epoch 2: 34.4% accurate
epoch 3: 44.9% accurate
epoch 4: 52.3% accurate
epoch 5: 56.4% accurate
epoch 6: 61.0% accurate
epoch 7: 65.7% accurate
epoch 8: 68.9% accurate
epoch 9: 71.1% accurate
epoch 10: 72.3% accurate
epoch 11: 73.7% accurate
epoch 12: 74.8% accurate
epoch 13: 76.5% accurate
epoch 14: 77.7% accurate
epoch 15: 78.0% accurate
epoch 16: 79.4% accurate
epoch 17: 80.1% accurate
epoch 18: 80.3% accurate
epoch 19: 81.2% accurate
epoch 20: 81.5% accurate
epoch 21: 82.0% accurate
epoch 22: 82.8% accurate
epoch 23: 83.0% accurate
epoch 24: 83.7% accurate
epoch 25: 83.6% accurate
epoch 26: 84.4% accurate
epoch 27: 84.3% accurate
epoch 28: 85.1% accurate
epoch 29: 84.6% accurate
epoch 30: 85.2% accurate


## AdaGrad
* adjusts learning rate based on parameters
* smaller learning rate for frequently occurring
* faster learning for infrequent
* modifies general learning rate at each time step for each parameter based on past gradients
  * weighti_t+1 = weighti_t - learning_rate/sqrt(Gi_t + epsilon) * gi_t
  * where Gi_t is the sum of squares of past gradients
  * epsilon is a small smoothing term to avoid dividing by zero (usually ~1e-8)
* main weakness is the ever accumulating gradient in the denominator which causes learning rate to shrink
* eventually stops learning

In [14]:
class Adagrad:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        # adding self variables for the weights/biases deltas
        # since we are tracking them over time now
        self.stacked_weights_delta = []
        self.stacked_biases_delta = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.stacked_weights_delta.append(np.zeros((layers[i-1], layers[i])))          
            self.stacked_biases_delta.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, learning_rate, batch_size, eps, test_data=None):   
        training_size = len(training_data)
        self.rate = learning_rate
        self.eps = eps
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):            
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            bias_grad = np.sum(rolling_delta * derivative, axis=0)
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.stacked_biases_delta[i] += bias_grad ** 2
            self.stacked_weights_delta[i] += weight_grad ** 2
            
            self.biases[i] += bias_grad * self.rate / np.sqrt(self.stacked_biases_delta[i] + self.eps)
            self.weights[i] += weight_grad * self.rate / np.sqrt(self.stacked_weights_delta[i] + self.eps)
            
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [45]:
ada = Adagrad([784, 50, 30, 10])
ada.train(training_data=train, time=30, learning_rate=.01, batch_size=50, eps=1e-8, test_data=test)

epoch 1: 25.7% accurate
epoch 2: 41.7% accurate
epoch 3: 49.8% accurate
epoch 4: 54.1% accurate
epoch 5: 58.1% accurate
epoch 6: 61.5% accurate
epoch 7: 65.3% accurate
epoch 8: 67.4% accurate
epoch 9: 69.3% accurate
epoch 10: 71.4% accurate
epoch 11: 72.8% accurate
epoch 12: 74.6% accurate
epoch 13: 76.3% accurate
epoch 14: 77.1% accurate
epoch 15: 78.4% accurate
epoch 16: 79.4% accurate
epoch 17: 79.9% accurate
epoch 18: 79.9% accurate
epoch 19: 80.6% accurate
epoch 20: 80.8% accurate
epoch 21: 81.2% accurate
epoch 22: 81.7% accurate
epoch 23: 82.4% accurate
epoch 24: 82.8% accurate
epoch 25: 82.8% accurate
epoch 26: 83.1% accurate
epoch 27: 83.5% accurate
epoch 28: 83.9% accurate
epoch 29: 84.1% accurate
epoch 30: 84.6% accurate


## AdaDelta
* adjusts the learning rate based on a window of past gradients
* also incorporates time decayed average of past deltas
* this makes it so that the parameter deltas have the same units as the parameters
* weights_t+1 = weights_t - RMS(delta_t-1)/RMS(gradient_t) * gradient_t

In [29]:
class Adadelta:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        # adding self variables for the weights/biases deltas
        # since we are tracking them over time now
        self.rms_weight_delta = []
        self.rms_biases_delta = []
        self.rms_weight_grad = []
        self.rms_biases_grad = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.rms_weight_delta.append(np.zeros((layers[i-1], layers[i])))          
            self.rms_biases_delta.append(np.zeros(layers[i]))
            self.rms_weight_grad.append(np.zeros((layers[i-1], layers[i])))          
            self.rms_biases_grad.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, decay, batch_size, eps, test_data=None):   
        training_size = len(training_data)
        self.decay = decay
        self.eps = eps
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):            
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            biases_grad = np.sum(rolling_delta * derivative, axis=0)
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.rms_biases_grad[i] = self.decay * self.rms_biases_grad[i] + (1-self.decay) * biases_grad**2
            self.rms_weight_grad[i] = self.decay * self.rms_weight_grad[i] + (1-self.decay) * weight_grad**2
            
            biases_delta = biases_grad * np.sqrt(self.rms_biases_delta[i] + self.eps) / np.sqrt(self.rms_biases_grad[i] + self.eps)
            weight_delta = weight_grad * np.sqrt(self.rms_weight_delta[i] + self.eps) / np.sqrt(self.rms_weight_grad[i] + self.eps)
            
            self.rms_biases_delta[i] = self.decay * self.rms_biases_delta[i] + (1-self.decay) * biases_delta**2
            self.rms_weight_delta[i] = self.decay * self.rms_weight_delta[i] + (1-self.decay) * weight_delta**2
            
            self.biases[i] += biases_delta
            self.weights[i] += weight_delta
                       
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [44]:
ad = Adadelta([784, 50, 30, 10])
ad.train(training_data=train, time=30, decay=.9, batch_size=50, eps=1e-8, test_data=test)

epoch 1: 11.1% accurate
epoch 2: 17.8% accurate
epoch 3: 32.5% accurate
epoch 4: 43.0% accurate
epoch 5: 49.8% accurate
epoch 6: 54.6% accurate
epoch 7: 58.5% accurate
epoch 8: 61.5% accurate
epoch 9: 65.1% accurate
epoch 10: 66.6% accurate
epoch 11: 67.4% accurate
epoch 12: 69.2% accurate
epoch 13: 70.8% accurate
epoch 14: 72.5% accurate
epoch 15: 72.9% accurate
epoch 16: 73.9% accurate
epoch 17: 75.3% accurate
epoch 18: 76.2% accurate
epoch 19: 76.6% accurate
epoch 20: 77.4% accurate
epoch 21: 77.7% accurate
epoch 22: 78.4% accurate
epoch 23: 78.7% accurate
epoch 24: 80.3% accurate
epoch 25: 80.5% accurate
epoch 26: 80.5% accurate
epoch 27: 81.1% accurate
epoch 28: 81.6% accurate
epoch 29: 81.8% accurate
epoch 30: 81.8% accurate


## RMSprop
* very similar to AdaDelta
* developed around the same time but no association
* uses a learning rate instead of incorporating the decayed accumulated deltas
* weights_t+1 = weights_t - learning_rate/RMS(gradient_t) * gradient_t

In [37]:
class RMSprop:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        self.rms_weight_grad = []
        self.rms_biases_grad = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.rms_weight_grad.append(np.zeros((layers[i-1], layers[i])))          
            self.rms_biases_grad.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, decay, learning_rate, batch_size, eps, test_data=None):   
        training_size = len(training_data)
        self.decay = decay
        self.eps = eps
        self.rate = learning_rate
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):            
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            biases_grad = np.sum(rolling_delta * derivative, axis=0)
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            self.rms_biases_grad[i] = self.decay * self.rms_biases_grad[i] + (1-self.decay) * biases_grad**2
            self.rms_weight_grad[i] = self.decay * self.rms_weight_grad[i] + (1-self.decay) * weight_grad**2
            
            biases_delta = biases_grad * self.rate / np.sqrt(self.rms_biases_grad[i] + self.eps)
            weight_delta = weight_grad * self.rate / np.sqrt(self.rms_weight_grad[i] + self.eps)
            
            self.biases[i] += biases_delta
            self.weights[i] += weight_delta
                       
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [43]:
rms = RMSprop([784, 50, 30, 10])
rms.train(training_data=train, time=30, decay=.9, learning_rate=.01, batch_size=50, eps=1e-8, test_data=test)

epoch 1: 84.6% accurate
epoch 2: 88.6% accurate
epoch 3: 90.4% accurate
epoch 4: 91.2% accurate
epoch 5: 90.9% accurate
epoch 6: 91.4% accurate
epoch 7: 91.9% accurate
epoch 8: 92.3% accurate
epoch 9: 92.4% accurate
epoch 10: 92.5% accurate
epoch 11: 93.3% accurate
epoch 12: 92.9% accurate
epoch 13: 93.1% accurate
epoch 14: 93.5% accurate
epoch 15: 93.6% accurate
epoch 16: 93.2% accurate
epoch 17: 93.3% accurate
epoch 18: 93.3% accurate
epoch 19: 94.3% accurate
epoch 20: 93.9% accurate
epoch 21: 93.6% accurate
epoch 22: 93.8% accurate
epoch 23: 93.5% accurate
epoch 24: 93.5% accurate
epoch 25: 94.2% accurate
epoch 26: 93.3% accurate
epoch 27: 94.0% accurate
epoch 28: 94.0% accurate
epoch 29: 93.8% accurate
epoch 30: 93.5% accurate


## Adam
* Incorporates adaptive learning rate (via decaying average of past squared gradients a la RMSprop/Adadelta) and momentum (via decaying average of past gradient)
    * m_t = decay1 * m_t-1 + (1 - decay1) * gradient_t
    * v_t = decay2 * v_t-1 + (1 - decay2) * gradient_t ^ 2
* m_t is an estimate of the gradient mean and v_t is an estimate of the gradient variance
* because both are set to 0 initial they are biased towards zero
* compute bias-corrected estimates to counteract:
  * m_t = m_t / (1 - decay1)
  * v_t = v_t / (1 - decay2)
* use corrected values to compute delta
  * weights_t+1 = weights_t - learning_rate * m_t / (sqrt(v_t) + eps)
* authors who came up with Adam suggested:
  * decay1 = 0.9
  * decay2 = 0.999
  * eps = 1e-8

In [51]:
class Adam:
    def __init__(self, layers):
        self.weights = []
        self.biases = []
        self.l = len(layers)
        self.layers = layers
        
        # add variables for gradient mean and variance estimates
        self.weight_m = []
        self.biases_m = []
        self.weight_v = []
        self.biases_v = []
        
        
        for i in range(1, self.l):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
            
            # set inital deltas to be zero
            self.weight_m.append(np.zeros((layers[i-1], layers[i])))          
            self.biases_m.append(np.zeros(layers[i]))
            self.weight_v.append(np.zeros((layers[i-1], layers[i])))          
            self.biases_v.append(np.zeros(layers[i]))
        
    def train(self, training_data, time, mdecay, vdecay, learning_rate, batch_size, eps, test_data=None):   
        training_size = len(training_data)
        self.m_decay = mdecay
        self.v_decay = vdecay
        self.eps = eps
        self.rate = learning_rate
        
        for i in range(time):
            np.random.shuffle(training_data)
            mini_x = [training_data[i:i+batch_size, :self.layers[0]] for i in range(0, training_size, batch_size)]
            mini_y = [training_data[i:i+batch_size, self.layers[0]:] for i in range(0, training_size, batch_size)]
            for x, y in zip(mini_x, mini_y):
                self.feedforward(x)
                self.backprop(y)
            if test_data is not None:
                print(f'epoch {i+1}: {round(self.evaluate(test_data)*100,2)}% accurate')
            
    def feedforward(self, x):
        self.zs = []
        self.activations = [x]
        for i in range(self.l-1):            
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.zs.append(z)
            
            a = sigmoid(z)
            self.activations.append(a)
            
    def backprop(self, y):        
        rolling_delta = y - self.activations[-1]
        for i in range(-1, -(self.l), -1):
            derivative = sigmoid_prime(self.zs[i])
            
            biases_grad = np.sum(rolling_delta * derivative, axis=0)
            weight_grad = np.dot(self.activations[i-1].T, rolling_delta * derivative)
            
            # calculate exponentially decaying average of past gradients & squared gradients 
            self.biases_m[i] = self.m_decay * self.biases_m[i] + (1-self.m_decay) * biases_grad
            self.weight_m[i] = self.m_decay * self.weight_m[i] + (1-self.m_decay) * weight_grad
            self.biases_v[i] = self.v_decay * self.biases_v[i] + (1-self.v_decay) * biases_grad**2
            self.weight_v[i] = self.v_decay * self.weight_v[i] + (1-self.v_decay) * weight_grad**2
            
            # correct for 0 bias
            bias_mhat = self.biases_m[i]/(1-self.m_decay)
            weight_mhat = self.weight_m[i]/(1-self.m_decay)
            bias_vhat = self.biases_v[i]/(1-self.v_decay)
            weight_vhat = self.weight_v[i]/(1-self.v_decay)
            
            self.biases[i] += bias_mhat * self.rate / (np.sqrt(bias_vhat) + self.eps)
            self.weights[i] += weight_mhat * self.rate / (np.sqrt(weight_vhat) + self.eps)
                       
            rolling_delta = np.dot(rolling_delta * derivative, self.weights[i].T)

        
    def evaluate(self, test_data):
        self.feedforward(test_data[:,:self.layers[0]])
        digit_yhat = np.argmax(self.activations[-1], axis=1)
        digit_y = np.argmax(test_data[:,self.layers[0]:], axis=1)
        return sum([yhat == y for (yhat, y) in zip(digit_yhat, digit_y)]) / len(test_data)

In [52]:
adam = Adam([784, 50, 30, 10])
adam.train(training_data=train, time=30, mdecay=.9, vdecay=.999, learning_rate=.01, batch_size=50, eps=1e-8, test_data=test)

epoch 1: 62.9% accurate
epoch 2: 65.2% accurate
epoch 3: 71.5% accurate
epoch 4: 78.5% accurate
epoch 5: 79.4% accurate
epoch 6: 80.4% accurate
epoch 7: 82.6% accurate
epoch 8: 91.4% accurate
epoch 9: 91.0% accurate
epoch 10: 91.9% accurate
epoch 11: 91.7% accurate
epoch 12: 92.1% accurate
epoch 13: 91.8% accurate
epoch 14: 92.3% accurate
epoch 15: 91.9% accurate
epoch 16: 92.3% accurate
epoch 17: 91.4% accurate
epoch 18: 92.2% accurate
epoch 19: 92.1% accurate
epoch 20: 92.3% accurate
epoch 21: 92.1% accurate
epoch 22: 92.0% accurate
epoch 23: 92.4% accurate
epoch 24: 92.7% accurate
epoch 25: 92.5% accurate
epoch 26: 92.8% accurate
epoch 27: 92.4% accurate
epoch 28: 93.3% accurate
epoch 29: 92.4% accurate
epoch 30: 93.1% accurate


## AdaMax
* updates Adam using infinity norm
* from Adam: 
 * v_t = decay2 * v_t-1 + (1 - decay2) * gradient_t^2
* using infinity norm:
 * u_t = decay2^inf * v_t-1 + (1 - decay2^inf) * gradient_t^inf
 * u_t = max(decay2 * v_t-1, gradient_t)
* weights_t+1 = weights_t - learning_rate/u_t * mhat_t

## Nadam
* Adam + NAG

## AMSGrad
* In some situations the exponential average of past squared gradients causes poor generalization
* In settings where Adam converges to a suboptimal solution, it has been observed that some minibatches provide large and informative gradients, but as these minibatches only occur rarely, exponential averaging diminishes their influence, which leads to poor convergence. 
* AMSGrad fixes this issue by using the max past squared gradient instead of average
* calculate v_t same as in Adam
* then instead of the bias corrected vhat_t from Adam
* vhat_t = max(vhat_t-1, v_t)
* results in non-increasing step size
* authors of this method saw improved performance (compared to Adam) on small datasets